# NFL Scores and Betting Data
## Grant Cloud

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from helpers import vis_null

In [ ]:
df = pd.read_csv('data/nfl_spread_covers.csv')

In [18]:
# calculate confusion matrix of cover    
def calc_cover(row):
    if row.favorite == row.team_home:
        diff = (row.score_home + row.spread_favorite) - row.score_away
        if diff > 0:
            return 'home_favorite'
        elif diff < 0:
            return 'away_underdog'
        else:
            return 'push'
    elif row.favorite == row.team_away:
        diff = row.score_home - (row.score_away + row.spread_favorite)
        if diff > 0:
            return 'home_underdog'
        elif diff < 0:
            return 'away_favorite'
        else:
            return 'push'

In [19]:
df['cover'] = df.apply(calc_cover, axis=1)

In [20]:
# num pushes for home team favorites
df[(df.favorite == df.team_home)&(((df.score_home + df.spread_favorite) - df.score_away) == 0)].cover.value_counts()

push    21
Name: cover, dtype: int64

In [21]:
# num pushes for away team favorites
df[(df.favorite == df.team_away)&((df.score_home - (df.score_away + df.spread_favorite)) == 0)].cover.value_counts()

push    16
Name: cover, dtype: int64

In [22]:
df.cover.value_counts()

away_underdog    320
home_favorite    299
away_favorite    171
home_underdog    170
push              37
Name: cover, dtype: int64

In [23]:
df.shape

(1024, 12)

In [24]:
# create confusion matrix of % values with push (ask dylan)
covSer = df.cover.value_counts()
au = covSer[0] / ( covSer[0] + covSer[1]  + 21)
hf = covSer[1] / ( covSer[0] + covSer[1] + 21)
af = covSer[2] / ( covSer[2] + covSer[3] + 16)
hu = covSer[3] / ( covSer[3] + covSer[3] + 16)
push = covSer[4] / (covSer[0] + covSer[1] + covSer[2] + covSer[3] + covSer[4])

In [25]:
pd.Series([au,hf,af,hu,push], index=df.cover.value_counts().index)

away_underdog    0.500000
home_favorite    0.467187
away_favorite    0.478992
home_underdog    0.477528
push             0.037111
dtype: float64

In [26]:
# not including pushes
au = covSer[0] / ( covSer[0] + covSer[1])
hf = covSer[1] / ( covSer[0] + covSer[1])
af = covSer[2] / ( covSer[2] + covSer[3])
hu = covSer[3] / ( covSer[3] + covSer[3])
push = covSer[4] / (covSer[0] + covSer[1] + covSer[2] + covSer[3] + covSer[4])

In [27]:
pd.Series([au,hf,af,hu,push], index=df.cover.value_counts().index)

away_underdog    0.516963
home_favorite    0.483037
away_favorite    0.501466
home_underdog    0.500000
push             0.037111
dtype: float64

In [28]:
# is it worth it? long run % profit 

# lmao no